# Introduction to biobase

Bulker comes with a core manifest called `biobase` that includes more than 50 common bioinformatics tools. Biobase is useful for everyday interactive analysis, basic pipelines, and also as a starting point for more complicated manifests. In this tutorial we'll show how to activate biobase and use it for some interactive RNA-seq analysis, and to run a pipeline. I assume you've already gone through the [install and configure](install.md) instructions.


## Loading the biobase crate

Let's load the biobase crate, which is available on the bulker registry. First I'm initializing an empty bulker config for this tutorial.

In [1]:
export BULKERCFG="biobase_example/bulker_config.yaml"
rm $BULKERCFG
bulker init -c $BULKERCFG

Guessing container engine is docker.
Wrote new configuration file: biobase_example/bulker_config.yaml


In [2]:
bulker load biobase -f $HOME/code/hub.bulker.io/bulker/biobase.yaml

Bulker config: /home/nsheff/code/bulker/docs_jupyter/biobase_example/bulker_config.yaml
Importing crate 'bulker/alpine:default' from '/home/nsheff/bulker_crates/bulker/alpine/default'.
Importing crate 'bulker/coreutils:default' from '/home/nsheff/bulker_crates/bulker/coreutils/default'.
Loading manifest: 'bulker/biobase:default'. Activate with 'bulker activate bulker/biobase:default'.
Commands available: aws, ascp, bamtools, bedClip, bedCommonRegions, bedGraphToBigWig, bedIntersect, bedItemOverlapCount, bedops, bedPileUps, bedToBigBed, bedtools, bigWigAverageOverBed, bigWigCat, bigWigSummary, bismark, bissnp, blast, bowtie2, bowtie, bsmap, bwa, cellranger, cufflinks, curl, cutadapt, faSplit, fastq-dump, fasterq-dump, vdb-config, fastqc, gatk, gt, hisat2, homer, kallisto, khmer, liftOver, macs2, mashmap, picard, pigz, prefetch, R, repeatmasker, rg, Rscript, sambamba, salmon, samtools, segway, seqkit, seqtk, skewer, samblaster, STAR, tabix, trim_galore, trimmomatic, vep, wigToBigWig


You can see this crate offers many common bioinformatics tools, like `samtools` and `bowtie2`. You can see this crate in your list of available crates:

In [3]:
bulker list

Available crates:
bulker/biobase:default -- /home/nsheff/bulker_crates/bulker/biobase/default


: 1

Before activating the crate, these commands are not available because they are not installed natively on this sytem:

In [4]:
samtools --version


Command 'samtools' not found, but can be installed with:

sudo apt install samtools



: 127

In [5]:
kallisto


Command 'kallisto' not found, but can be installed with:

sudo apt install kallisto



: 127

So, we activate the crate. In a shell you just type:

```
bulker activate biobase
```

Due to a limitation with jupyter, because bulker spawns a new shell by default, you have to things a bit differently...there 2 ways to use bulker with a jupyter note book:

1. You can simply activate the crate before starting the notebook (the easier way).
2. You can use this `eval` trick below to activate the crate in the existing jupyter shell.

Either way should have the same affect as typing `bulker activate biobase` outside of jupyter.

In [4]:
eval "$(bulker activate -e -p biobase)"

Bulker config: /home/nsheff/pCloudSync/env/bulker_config/zither.yaml
Activating bulker crate: biobase


It's worth emphasizing: **that single line of code is all you need to do to "install" dozens of common bioinformatics tools**. Now, in just a few minutes you've given yourself the ability to create a portable computing environment instantly, on demand. 

Now we can inspect what commands are made available by this crate:

In [7]:
bulker inspect

Bulker config: /home/nsheff/code/bulker/docs_jupyter/biobase_example/bulker_config.yaml
Bulker manifest: bulker/biobase
Crate path: /home/nsheff/bulker_crates/bulker/biobase/default
Available commands: ['cellranger', 'fasterq-dump', 'cksum', 'hisat2', 'paste', 'chmod', 'aws', 'picard', 'arch', 'tail', 'unexpand', 'liftOver', 'cutadapt', 'expr', 'rm', 'STAR', 'base32', 'runcon', 'timeout', 'gt', 'samtools', 'nohup', 'rmdir', 'kallisto', 'sleep', 'vdir', 'bowtie2', 'pigz', 'samblaster', 'echo', 'base64', 'bwa', 'which', 'bash', 'shuf', 'homer', 'trimmomatic', 'salmon', 'stty', 'fastqc', 'cp', 'chgrp', 'ptx', 'blast', 'mkfifo', 'mashmap', 'seq', 'du', 'bigWigCat', 'logname', 'factor', 'vep', 'dircolors', 'bedCommonRegions', 'pwd', 'bigWigAverageOverBed', 'expand', 'sh', 'mkdir', 'whoami', 'bedops', 'R', 'bedPileUps', 'tsort', 'users', 'prefetch', 'fmt', 'truncate', 'csplit', 'repeatmasker', 'bissnp', 'gatk', 'uptime', 'tee', 'touch', 'false', 'od', 'mv', 'numfmt', 'bismark', 'seqtk', 'fas

Any of these commands can be run as if they are native, but they will actually be running in containers.

In [8]:
samtools --version

samtools 1.10
Using htslib 1.10.2
Copyright (C) 2019 Genome Research Ltd.


In [9]:
kallisto

kallisto 0.46.2

Usage: kallisto <CMD> [arguments] ..

Where <CMD> can be one of:

    index         Builds a kallisto index 
    quant         Runs the quantification algorithm 
    bus           Generate BUS files for single-cell data 
    pseudo        Runs the pseudoalignment step 
    merge         Merges several batch runs 
    h5dump        Converts HDF5-formatted results to plaintext
    inspect       Inspects and gives information about an index
    version       Prints version information
    cite          Prints citation information

Running kallisto <CMD> without arguments prints usage information for <CMD>



: 1

In [ ]:
cutadapt --version

## Running an analysis

Now that we've proven we can run each of these commands, let's put them all together and run a whole pipeline. Since the environment we've just activated has all those commands available as if they were installed natively, all we have to do is run a bunch of commands in succession and they will automatically run in individual containers.


Let's grab some reads from SRA:

In [1]:
wget  https://sra-download.ncbi.nlm.nih.gov/traces/sra10/SRR/011497/SRR11773889

--2020-07-10 14:36:20--  https://sra-download.ncbi.nlm.nih.gov/traces/sra10/SRR/011497/SRR11773889
Resolving sra-download.ncbi.nlm.nih.gov (sra-download.ncbi.nlm.nih.gov)... 130.14.250.25, 130.14.250.24, 165.112.9.231
Connecting to sra-download.ncbi.nlm.nih.gov (sra-download.ncbi.nlm.nih.gov)|130.14.250.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15095418 (14M) [application/octet-stream]
Saving to: ‘SRR11773889’

SRR11773889         100%[===================>]  14.40M  49.3KB/s    in 2m 31s  

2020-07-10 14:38:53 (97.4 KB/s) - ‘SRR11773889’ saved [15095418/15095418]



In [8]:
fasterq-dump -L info SRR11773889 -O raw_reads

spots read      : 742,462
reads read      : 742,462
reads written   : 742,462


Now we want to try to align these. We'll use refgenie to easily pull some bowtie2 indexes. Refgenie is included in the biobase.

In [18]:
export REFGENIE="biobase_example/refgenie.yaml"
refgenie init -c $REFGENIE -s http://staging.refgenomes.databio.org
refgenie pull t7/bowtie2_index

You should consider upgrading via the 'pip install --upgrade pip' command.
Initializing refgenie genome configuration
Wrote new refgenie genome configuration file: peppro_example/refgenie.yaml
'hs38d1/fasta:default' archive size: 1.7MB
hs38d1/fasta:default: 1.74MB [00:00, 28.8MB/s]
Download complete: /home/nsheff/code/bulker/docs_jupyter/peppro_example/hs38d1/fasta__default.tgz
Extracting asset tarball and saving to: /home/nsheff/code/bulker/docs_jupyter/peppro_example/hs38d1/fasta/default
Default tag for 'hs38d1/fasta' set to: default
'hs38d1/bowtie2_index:default' archive size: 8.8MB
hs38d1/bowtie2_index:default: 9.22MB [00:00, 21.1MB/s]                          
Download complete: /home/nsheff/code/bulker/docs_jupyter/peppro_example/hs38d1/bowtie2_index__default.tgz
Extracting asset tarball and saving to: /home/nsheff/code/bulker/docs_jupyter/peppro_example/hs38d1/bowtie2_index/default
Default tag for 'hs38d1/bowtie2_index' set to: default


In [5]:
bowtie2 -x `refgenie seek rCRSd/bowtie2_index` raw_reads/SRR11773889.fastq -S aligned.sam

742462 reads; of these:
  742462 (100.00%) were unpaired; of these:
    742461 (100.00%) aligned 0 times
    0 (0.00%) aligned exactly 1 time
    1 (0.00%) aligned >1 times
0.00% overall alignment rate


In [7]:
bowtie2 -x `refgenie seek t7/bowtie2_index` raw_reads/SRR11773889.fastq -S aligned.sam

742462 reads; of these:
  742462 (100.00%) were unpaired; of these:
    49655 (6.69%) aligned 0 times
    689848 (92.91%) aligned exactly 1 time
    2959 (0.40%) aligned >1 times
93.31% overall alignment rate


This pipeline also requires reference genome assets that are managed by [refgenie](http://refgenie.databio.org). If you don't already have an initialized refgenie config, you can easily initialize one like this:

Now execute the example code to run it.

In [19]:
./peppro_example/peppro/pipelines/peppro.py \
  --sample-name test \
  --genome hs38d1 \
  --input peppro_example/peppro/examples/data/test_r1.fq.gz \
  --single-or-paired single \
  -O peppro_example/output/

### Pipeline run code and environment:

*              Command:  `./peppro_example/peppro/pipelines/peppro.py --sample-name test --genome hs38d1 --input peppro_example/peppro/examples/data/test_r1.fq.gz --single-or-paired single -O peppro_example/output/`
*         Compute host:  puma
*          Working dir:  /home/nsheff/code/bulker/docs_jupyter
*            Outfolder:  /home/nsheff/code/bulker/docs_jupyter/peppro_example/output/test/
*  Pipeline started at:   (10-21 11:29:03) elapsed: 0.0 _TIME_

### Version log:

*       Python version:  2.7.12
*          Pypiper dir:  `/home/nsheff/.local/lib/python2.7/site-packages/pypiper`
*      Pypiper version:  0.12.0
*         Pipeline dir:  `/home/nsheff/code/bulker/docs_jupyter/peppro_example/peppro/pipelines`
*     Pipeline version:  0.8.1
*        Pipeline hash:  05095a2cc78a2e210916f215dd0828940b894a6f
*      Pipeline branch:  * dev
*        Pipeline date:  2019-10-21 11:21:23 -0400

### Arguments passed to pipeline:

*           `TSS_na


> `Total_efficiency`	3.32	PEPPRO	_RES_

> `Read_depth`	1.26	PEPPRO	_RES_

### Compress all unmapped read files (10-21 11:29:11) elapsed: 4.0 _TIME_

Target to produce: `/home/nsheff/code/bulker/docs_jupyter/peppro_example/output/test/aligned_hs38d1/test_temp.bam.bai`  

> `samtools index /home/nsheff/code/bulker/docs_jupyter/peppro_example/output/test/aligned_hs38d1/test_temp.bam` (21800)
<pre>
</pre>
Command completed. Elapsed time: 0:00:01. Running peak memory: 0.057GB.  
  PID: 21800;	Command: samtools;	Return code: 0;	Memory used: 0.019GB


> `samtools idxstats /home/nsheff/code/bulker/docs_jupyter/peppro_example/output/test/aligned_hs38d1/test_temp.bam | grep -we 'chrM' -we 'chrMT' -we 'M' -we 'MT' -we 'rCRSd' -we 'rCRSd_3k'| cut -f 3`

> `samtools stats /home/nsheff/code/bulker/docs_jupyter/peppro_example/output/test/aligned_hs38d1/test_sort.bam | grep '^SN' | cut -f 2- | grep 'maximum length:' | cut -f 2-`

### Calculate NRF, PBC1, and PBC2 (10-21 11:29:13) elapsed: 2.0 _TIME_


<pre>
Registering input file: '/home/nsheff/code/bulker/docs_jupyter/peppro_example/output/test/aligned_hs38d1/test_minus.bam'
Temporary files will be stored in: 'tmp_test_minus_cuttrace_ljeMBw'
Processing with 1 cores...
Reduce step (merge files)...
Merging 124 files into output file: '/home/nsheff/code/bulker/docs_jupyter/peppro_example/output/test/signal_hs38d1/test_minus_body_0-mer.bw'
</pre>
Command completed. Elapsed time: 0:01:23. Running peak memory: 0.068GB.  
  PID: 31872;	Command: ./peppro_example/peppro/tools/bamSitesToWig.py;	Return code: 0;	Memory used: 0.057GB

Starting cleanup: 5 files; 4 conditional files for cleanup

Cleaning up flagged intermediate files. . .

Cleaning up conditional list. . .

### Pipeline completed. Epilogue
*        Elapsed time (this run):  0:03:01
*  Total elapsed time (all runs):  0:02:59
*         Peak memory (this run):  0.0675 GB
*        Pipeline completed time: 2019-10-21 11:32:04


The pipeline has now completed successfully and we can explore the results:

In [20]:
tree peppro_example/output/

peppro_example/output/
└── test
    ├── aligned_hs38d1
    │   ├── test_fail_qc.bam
    │   ├── test_minus.bam
    │   ├── test_minus.bam.bai
    │   ├── test_plus.bam
    │   ├── test_plus.bam.bai
    │   ├── test_sort.bam
    │   ├── test_sort.bam.bai
    │   └── test_unmap.bam
    ├── cutadapt
    │   └── test_cutadapt.txt
    ├── fastq
    ├── fastqc
    ├── objects.tsv
    ├── PEPPRO_cleanup.sh
    ├── PEPPRO_commands.sh
    ├── PEPPRO_completed.flag
    ├── PEPPRO_log.md
    ├── PEPPRO_profile.tsv
    ├── QC_hs38d1
    │   └── test_bamQC.tsv
    ├── raw
    │   └── test.fastq.gz -> /home/nsheff/code/bulker/docs_jupyter/peppro_example/peppro/examples/data/test_r1.fq.gz
    ├── signal_hs38d1
    │   └── test_minus_body_0-mer.bw
    └── stats.tsv

8 directories, 19 files


We've successfully run a complete pipeline without having to install any of the software that runs the commands in the workflow. We're also able to interactively explore the environment that ran the workflow.


## Conclusion

That's basically it. If you're a workflow developer, all you need to do is [write your own manifest](manifest.md) and distribute it with your workflow; in 3 lines of code, users will be able to run your workflow using modular containers, using the container engine of their choice.

